In [48]:
# import modules
import os
import shutil
from datetime import date

In [1]:
'''Copying and renaming files'''

def bidsify(origpath, destpath, n_sessions=2, scan_types=None, detect_size=True, log_changes=True, log_name='CHANGES', verbose=False):
    
    
    # NOTE: mapping old names to new names by passing a dict is not currently supported (scan_names kwarg).
    # detect_size must be True (default) to work. Passing custom file size ranges not currently supported (file_sizes kwarg)
    
    # convert to absolute paths if relative path is passed
    origpath_abs = os.path.abspath(origpath)
    destpath_abs = os.path.abspath(destpath)
    
    # dict to hold unsuccessfully handled files with errors
    problem_files = {}
    
    # default output folder names
    if scan_types is None:
        scan_types = ['anat','func','log']
    
    # for name mapping based on file size
    if detect_size:    #and scan_names is None:
        method = 'size'
        
    else:
        raise ValueError('detect_size must be True. Mapping by other file attributes not currently supported.')
    
    # for name mapping based on passed names
#     elif not detect_size and scan_names is None:
#         # use default dictionary of scan types
#         scan_names = {
#             'mprage' : 'T1w',
#             'bold1' : 'task-rest_run-01_bold',
#             'bold2' : 'task-mcr_run-02_bold',
#             'bold3' : 'task-swm_run-03_bold',
#             'bold4' : 'task-dd_run-04_bold',
#             'bold5' : 'task-rest_run-05_bold'
#             }
#         method = 'name'
    
#     elif detect_size and scan_names:
#         raise ValueError('Cannot map by both scan_names and detect_size')
        
#     if scan_names and type(scan_names) is not dict:
#         raise TypeError('scan_names must be a dict')

    for i, (root, dirs, files) in enumerate(os.walk(origpath_abs, topdown=True)):
        # do not recurse into the 'DONTUSE' folder
        dirs[:] = [d for d in dirs if not d.startswith('DONTUSE')]
        
        # create new directory structure
        if i == 0:
            [os.makedirs(destpath_abs+'/'+direc.split('_')[0]+'/ses-'+str(ses+1)+'/'+scantype, exist_ok=True) for direc in dirs for scantype in scan_types for ses in range(n_sessions)]
        
        # move and rename files
        else:
            file_list = [f for f in files if not f.startswith('.')]
            if file_list:
                if method == 'size':
                    path_map, prob_fs = _rename_size(file_list, root, n_sessions, destpath_abs, log_changes, log_name, verbose)
                    
                    for old_filepath, new_filepath in path_map.items():
                        shutil.copy(old_filepath, new_filepath)

                        if verbose:
                            print('moved ' + old_filepath + ' to ' + new_filepath)
                            
                    if prob_fs:
                        problem_files.update({subID: prob_fs})
                                
#                 elif method == 'name':
#                     for file in file_list:
#                         old_filepath = os.path.join(root, file)
#                         new_filepath, problem_file = _rename_dict(file, root, n_sessions, scan_names, destpath_abs, log_changes, log_name, verbose)

#                         if new_filepath is not None:
#                             # move and rename
#                             shutil.copy(old_filepath, new_filepath)

#                             if verbose:
#                                 print('moved ' + old_filepath + ' to ' + new_filepath)

#                         else:
#                             problem_files.append(problem_file)       
                    
                    
    if problem_files:
        print('The following files were not successfully converted: ' + problem_files)

SyntaxError: invalid syntax (<ipython-input-1-18672f7d9878>, line 58)

In [5]:
# def _rename_dict(file, root, n_sessions, scan_names, destpath_abs, log_changes, log_name, verbose):

#     # viarable to track unsuccessfully renamed files
#     problem_file = None

#     old_path = os.path.join(root, file)
#     base, ext = os.path.splitext(file)
#     splitpath = root.split('/')

#     # get subject ID
#     sub = splitpath[-2].split('_')[0]

#     # get session number
#     ses_number = splitpath[-2].split('_')[1]
#     if int(ses_number) <= n_sessions:
#         session = 'ses-'+ses_number

#     else:
#         print('unrecognized session number \'' + ses_number + '\' for subID ' + sub)
#         problem_file = file

#     # get scan type (or log)
#     if splitpath[-1] == 'ANATOMY':
#         runtype = 'anat'

#     elif splitpath[-1] == 'FUNCTIONAL':
#         runtype = 'func'

#     elif splitpath[-1] == 'LOG':
#         runtype = 'LOG'

#     else:
#         print('unrecognized scan or log folder ' + splitpath[-1] + ' for subID ' + sub)
#         problem_file = file

        
#     # format scan name (or preserve name of log file)
#     if base in scan_names:
#         new_name = scan_names[base]

#     elif ext == '.log':
#         new_name = base

#     else:
#         print('unrecognized scan name ' + base + ' for file ' + file)
#         problem_file = file

#     try:
#         if runtype != 'LOG':
#             new_path = os.path.join(destpath_abs, sub, session, runtype, sub+'_'+new_name+ext)
#         else:
#             new_path = os.path.join(destpath_abs, sub, session, runtype, new_name+ext)
            
#         if log_changes:
#             _writelog(log_name, old_path, destpath_abs, sub, session, new_name, ext, verbose)

#     except NameError:
#         new_path = None

    
#     return new_path, problem_file

In [ ]:
'''Defines naming scheme for files based on size'''

def _rename_size(file_list, root, n_sessions, destpath_abs, log_changes, log_name, verbose):
    
    old_fps = [os.path.join(root,file) for file in file_list]
    path_maps = dict.fromkeys(old_fps)
    sizes_dict = {os.stat(file).st_size: file for file in old_fps}
    prob_fs = {}
    splitpath = root.split('/')
    
    # get subject ID
    sub = splitpath[-2].split('_')[0]
    
    # get session number
    ses_number = splitpath[-2].split('_')[1].lstrip('0')
    if int(ses_number) <= n_sessions:
        session = 'ses-'+ses_number
        
    else:
        prob_fs['']= 'Unrecognized session number: ' + ses_number
        return path_maps, prob_fs
        
    
    # deal with anatomical scans
    if splitpath[-1] == 'ANATOMY':
        # expected size for mprage files (bytes)
        right_size = 28836192
        # use file that matches expected size, or if none do, use one closest to expected size
        best_scan = sizes_dict[right_size] if right_size in sizes_dict else sizes_dict[min(sizes_dict.keys(), key=lambda k: abs(k-right_size))]
        
        if right_size not in sizes_dict.keys():
            prob_fs[best_scan.split('/')[-1]] = 'No mprage of expected size. Used closest match: ' + best_scan.split('/')[-1])
        
        new_name = 'T1w.nii'
        path_maps[best_scan] = os.path.join(destpath_abs, sub, session, 'anat', sub+'_T1w.nii')
        
        
    # deal with functional scans
    elif splitpath[-1] == 'FUNCTIONAL':
        rests = []
        mcrs = []
        swms = []
        leftovers = []
        for size, old_file in sizes_dict.items():
            # expected size of resting state scan (+/- 2kb)
            if size in range(110590352, 110594352):
                rests.append(old_file)
                
            # expected size of mcr scan (+/- 2kb)    
            elif size in range(77412752, 77416752):
                mrcs.append(old_file)
            
            # expected size of swm scan (+/- 2kb)
            elif size in range(82942352, 82946352):
                swms.append(old_file) 
            else:
                leftovers.append(old_file)
        
        # map resting state scans
        if not rests:
            prob_fs['Resting state'] = 'No scan files matching expected size for Resting State.'
            
        elif len(rests) == 1:
            prob_fs['Resting state'] = 'Unable to determine which Resting State scan for file: ' + rests[0]
            
        elif len(rests) == 2:
            rest1, rest2 = sorted(rests)[0], sorted(rests)[1]
            path_maps[rest1] = os.path.join(destpath_abs, sub, session, 'func', sub+'_task-rest_run-01_bold.nii')
            path_maps[rest2] = os.path.join(destpath_abs, sub, session, 'func', sub+'_task-rest_run-05_bold.nii')
        else:
            prob_fs['Resting state'] = 'Unable to identify Resting State 1 vs 2 from choices: ' + rests[0]
                    
        # map mcr scans
        if mcrs:
            path_maps[mcrs[-1]] = os.path.join(destpath_abs, sub, session, 'func', sub+'_task-mcr_run-02_bold.nii')
        else:
            prob_fs['MCR'] = 'No scan files matching expected size for MCR'
            
        # map swm scans
        if swms:
            path_maps[swms[-1]] = os.path.join(destpath_abs, sub, session, 'func', sub+'_task-swm_run-03_bold.nii')
        else:
            prob_fs['SWM'] = 'No scan files matching expected size for SWM'
 
         
        if len(leftovers) == 1:
            path_maps[leftovers[0]] = os.path.join(destpath_abs, sub, session, 'func', sub+'_task-dd_run-04_bold.nii')
        else:
            prob_fs['DD'] = 'Unable to identify DD scan from choices: ' + str(leftovers)
            
    # deal with log files
    elif splitpath[-1] == 'LOG':
        runtype = 'LOG'
        # keep existing name
        for file in old_fps:
            path_maps[file] = os.path.join(destpath_abs, sub, session, 'log', sub+'_task-dd_run-04_bold.nii')
        
    else:
        prob_fs[''] = 'unrecognized scan or log folder: ' + splitpath[-1]
        return path_maps, prob_fs
    
    # create & write log file in destination dir with record of move
    if log_changes:
        _write_log(log_name, path_maps, destpath_abs, sub, session, verbose)
        
    # note any scan files not mapped...
    unmapped = [old_file for old_file, new_file in path_maps.items() if not new_file]
    if unmapped:
        prob_fs['Unmapped files'] = 'The following files were not moved: ' + str(unmapped)
        
    # ...and filter them from the dictionary
    filtered_path_maps = {k: v for k, v in path_maps.items() if v is not None}
        
    return filtered_path_maps, prob_fs

In [4]:
'''Writes/updates log of moving file'''

def _write_log(log_name, path_maps, destpath_abs, sub, session, verbose):
    
    filename = os.path.join(destpath_abs, sub, session, 'log', log_name + '.log')
    
    if os.path.exists(filename):
        mode = 'a'
    else:
        mode = 'w'
        
    with open(filename, mode) as f:
        f.write(date.today().strftime('%Y-%m-%d') + '\n')
        for old_path, new_path in path_maps.items():
            f.write('- ' + new_path + ' moved from ' + old_path + '\n')
    
    if verbose:
        print('wrote to log file ' + filename)

In [169]:
for i, (root, dirs, files) in enumerate(os.walk('/Users/paxtonfitzpatrick/Desktop/testorig/', topdown=True)):
    dirs[:] = [d for d in dirs if not d.startswith('DONTUSE')]
    file_list = [f for f in files if not f.startswith('.')]
    if file_list:
        print(root + '\t\t' + 'a')

/Users/paxtonfitzpatrick/Desktop/testorig/2022_1_asd/ANATOMY		a
/Users/paxtonfitzpatrick/Desktop/testorig/2022_1_asd/LOG		a
/Users/paxtonfitzpatrick/Desktop/testorig/2022_1_asd/FUNCTIONAL		a
/Users/paxtonfitzpatrick/Desktop/testorig/2020_1_asd/ANATOMY		a
/Users/paxtonfitzpatrick/Desktop/testorig/2020_1_asd/LOG		a
/Users/paxtonfitzpatrick/Desktop/testorig/2020_1_asd/FUNCTIONAL		a
/Users/paxtonfitzpatrick/Desktop/testorig/2019_1_asd/ANATOMY		a
/Users/paxtonfitzpatrick/Desktop/testorig/2019_1_asd/LOG		a
/Users/paxtonfitzpatrick/Desktop/testorig/2019_1_asd/FUNCTIONAL		a
/Users/paxtonfitzpatrick/Desktop/testorig/2021_1_asd/ANATOMY		a
/Users/paxtonfitzpatrick/Desktop/testorig/2021_1_asd/LOG		a
/Users/paxtonfitzpatrick/Desktop/testorig/2021_1_asd/FUNCTIONAL		a
/Users/paxtonfitzpatrick/Desktop/testorig/2023_1_asd/LOG		a
/Users/paxtonfitzpatrick/Desktop/testorig/2018_1_asd/ANATOMY		a
/Users/paxtonfitzpatrick/Desktop/testorig/2018_1_asd/LOG		a
/Users/paxtonfitzpatrick/Desktop/testorig/2018_1_asd

In [108]:
testdict = {'test'+str(i/2) : str(1-i) for i in range(8)}

In [106]:
basedict = {}

In [109]:
basedict.update({'subID2' : testdict})

In [110]:
basedict

{'subID': {'test0': '0',
  'test1': '1',
  'test2': '2',
  'test3': '3',
  'test4': '4',
  'test5': '5',
  'test6': '6',
  'test7': '7'},
 'subID2': {'test0.0': '1',
  'test0.5': '0',
  'test1.0': '-1',
  'test1.5': '-2',
  'test2.0': '-3',
  'test2.5': '-4',
  'test3.0': '-5',
  'test3.5': '-6'}}

In [115]:
'2011_1_1111'.split('_')[1].lstrip('0')

'1'

In [121]:
'1thingy'[0].isalpha()

False